In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('BBC News Train.csv',delimiter=',')
df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [4]:
#remove Articled
df=df.drop(['ArticleId'],axis=1)
df.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [5]:
pip install transformers torch datasets


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load the pre-trained BERT model for text classification (this model is fine-tuned for classification)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Adjust 'num_labels' based on your dataset


2024-12-02 12:58:35.614056: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 12:58:35.683179: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 12:58:36.103836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 12:58:37.438328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased 

In [7]:
! pip install datasets
from datasets import Dataset

# Convert the DataFrame to a Dataset object
dataset = Dataset.from_pandas(df)

# Tokenize the input text
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding='max_length', truncation=True)

# Apply the tokenize function
tokenized_datasets = dataset.map(tokenize_function, batched=True)

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


Map: 100%|██████████| 1490/1490 [00:09<00:00, 154.85 examples/s]


In [8]:
#read tokenised datadet
tokenized_datasets

Dataset({
    features: ['Text', 'Category', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1490
})

In [9]:
! pip install sklearn
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import AdamW
from sklearn.metrics import accuracy_score
import torch

# Assuming df is your DataFrame
dataset = Dataset.from_pandas(df)

# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.2)
tokenized_datasets = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# Tokenize the input text
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding='max_length', truncation=True)

# Apply the tokenize function
tokenized_datasets = tokenized_datasets.map(tokenize_function, batched=True)

# Identify all unique labels in the dataset
unique_labels = set(dataset['Category'])

# Create a mapping from labels to numerical IDs
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}

# Convert string labels to numerical labels
def encode_labels(examples):
    examples['Category'] = [label_to_id[label] for label in examples['Category']]
    return examples

# Apply the label encoding function
tokenized_datasets = tokenized_datasets.map(encode_labels, batched=True)

# Create DataLoader for training and validation
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=8, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
# Training loop
model.train()  # Set model to training mode
for epoch in range(3):  # Train for 3 epochs
    for batch in train_dataloader:
        optimizer.zero_grad()

        # Move input data to device (GPU/CPU)
        input_ids = torch.stack(batch['input_ids']).squeeze().to(device)
        attention_mask = torch.stack(batch['attention_mask']).squeeze().to(device)
        labels = torch.tensor(batch['Category']).to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Evaluate the model
    model.eval()  # Set model to evaluation mode
    preds = []
    labels = []
    for batch in eval_dataloader:
        with torch.no_grad():
            input_ids = torch.stack(batch['input_ids']).squeeze().to(device)
            attention_mask = torch.stack(batch['attention_mask']).squeeze().to(device)
            label = torch.tensor(batch['Category']).to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=-1)
            
            preds.extend(pred.cpu().numpy())
            labels.extend(label.cpu().numpy())

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    print(f"Epoch {epoch+1}, Accuracy: {accuracy}")

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
    ERROR: Command errored out with exit status 1:
     command: /usr/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-zqku2wfw/sklearn/setup.py'"'"'; __file__='"'"'/tmp/pip-install-zqku2wfw/sklearn/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-install-zqku2wfw/sklearn/pip-egg-info
         cwd: /tmp/pip-install-zqku2wfw/sklearn/
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
    rather than 'sklearn' for pip commands.
    
    Here is how to fix this error in the main use cases:
    - use 'pip install scikit-learn' rather 

Map: 100%|██████████| 298/298 [00:00<00:00, 72416.14 examples/s]
/home/lopamudra/.local/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/lopamudra/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/lopamudra/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered interna

ValueError: Expected input batch_size (512) to match target batch_size (8).